# Division reduced dataset to 30 seconds mp3 files

## Import

In [9]:
import os
import librosa
import numpy as np
import pandas as pd
from pydub import AudioSegment
import concurrent.futures
from tqdm import tqdm

## Processing audio files into 30-second files, removing silence, creating metadata

In [10]:
def process_track(row, audio_folder, output_folder, top_db=30, chunk_duration_sec=30):
    """
    Przetwarza jedną piosenkę (jeden wiersz metadanych):
      - Wczytuje plik audio (z audio_folder + row["PATH"])
      - Usuwa ciszę z początku i końca (librosa.effects.trim)
      - Dzieli audio na chunki 30 sek.
      - Każdy chunk zapisuje do MP3 w subfolderze output_folder
      - Zwraca listę słowników z metadanymi chunków,
        gdzie PATH = np. "57/1294657_chunk_0.mp3" (relatywnie do output_folder).
    """
    results = []
    
    track_id = row["TRACK_ID"]
    artist_id = row["ARTIST_ID"]
    album_id  = row["ALBUM_ID"]

    # Ścieżka do oryginalnego pliku audio
    # (zakładamy, że w metadanych 'PATH' jest relatywna ścieżka do audio_folder)
    audio_path = os.path.join(audio_folder, row["PATH"])

    try:
        # 1. Wczytaj audio
        audio, sr = librosa.load(audio_path, sr=None)
        
        # 2. Trim ciszy z początku i końca
        audio_trimmed, trimmed_indices = librosa.effects.trim(audio, top_db=top_db)
        start_trim, _ = trimmed_indices
        
        # 3. Oblicz liczbę chunków 30 sek (pomijamy końcówkę < 30s)
        chunk_length = chunk_duration_sec * sr
        total_samples = len(audio_trimmed)
        num_full_chunks = total_samples // chunk_length
        
        # 4. Podfolder docelowy wg 2 ostatnich cyfr track_id (np. "57")
        subfolder_name = str(track_id)[-2:].zfill(2)
        
        # 5. Iteracja po chunkach
        for i in range(num_full_chunks):
            start_idx = i * chunk_length
            end_idx   = start_idx + chunk_length
            chunk     = audio_trimmed[start_idx:end_idx]
            
            chunk_start_sec = (start_trim + start_idx) / sr
            chunk_end_sec   = (start_trim + end_idx)   / sr
            
            # Nazwa pliku MP3, np. "1294657_chunk_0.mp3"
            new_filename = f"{track_id}_chunk_{i}.mp3"
            
            # Relatywna ścieżka, np. "57/1294657_chunk_0.mp3"
            # Uwaga: używamy slash '/' by była ta sama na każdym OS.
            rel_path = f"{subfolder_name}/{new_filename}"
            
            # Ścieżka docelowa na dysku (output_folder + rel_path)
            dest_path = os.path.join(output_folder, subfolder_name, new_filename)
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            
            # 6. Konwersja -> int16 -> AudioSegment -> zapis MP3
            samples_int16 = (chunk * 32767).astype(np.int16)
            audio_segment = AudioSegment(
                samples_int16.tobytes(),
                frame_rate=sr,
                sample_width=2,
                channels=1
            )
            audio_segment.export(dest_path, format="mp3")
            
            # 7. Dodaj metadane chunku
            results.append({
                "CHUNK_ID": f"{track_id}_chunk_{i}",
                "TRACK_ID": track_id,
                "ARTIST_ID": artist_id,
                "ALBUM_ID": album_id,
                "PATH": rel_path,  # <--- TYLKO np. "57/1294657_chunk_0.mp3"
                "START_SEC": chunk_start_sec,
                "END_SEC": chunk_end_sec,
                "CHUNK_DURATION": chunk_duration_sec,
                "TAGS": row["TAGS"],
                "GENRE_LIST": row["genre_list"],
            })
    
    except Exception as e:
        print(f"[BŁĄD w process_track] {audio_path}: {e}")

    return results


def parallel_chunking_with_threads(
    metadata_csv,
    audio_folder,
    output_folder,
    output_metadata,
    top_db=30,
    chunk_duration_sec=30,
    max_workers=4
):
    """
    Równoległe przetwarzanie utworów z pliku CSV w wątkach:
     - Każdy wiersz = 1 plik audio do pocięcia
     - Zbiera metadane chunków i zapisuje do pliku CSV
     - Ścieżki chunków (PATH) zapisywane są RELATYWNIE,
       np. "57/1294657_chunk_0.mp3".
    """
    
    import concurrent.futures
    
    # 1. Wczytanie metadanych
    df = pd.read_csv(metadata_csv)
    
    # Sprawdź, czy mamy wymagane kolumny
    required_cols = ["TRACK_ID", "PATH", "TAGS", "genre_list", "ARTIST_ID", "ALBUM_ID"]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Brak wymaganej kolumny '{col}' w pliku {metadata_csv}.")
    
    # 2. Tworzymy docelowy folder
    os.makedirs(output_folder, exist_ok=True)

    # 3. Uruchamiamy ThreadPoolExecutor
    futures = []
    new_metadata = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        for row_tuple in df.itertuples(index=False):
            row_dict = row_tuple._asdict()
            future = executor.submit(
                process_track,
                row=row_dict,
                audio_folder=audio_folder,
                output_folder=output_folder,
                top_db=top_db,
                chunk_duration_sec=chunk_duration_sec
            )
            futures.append(future)
        
        # 4. Odbieramy wyniki i aktualizujemy pasek postępu
        for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing"):
            result = f.result()
            new_metadata.extend(result)
    
    # 5. Tworzymy finalny DataFrame i zapisujemy do CSV
    final_df = pd.DataFrame(new_metadata)
    final_df.to_csv(output_metadata, index=False)
    
    print(f"\nZapisano CSV z chunkami: {output_metadata}")
    print(f"Liczba chunków: {len(final_df)}")

In [11]:
metadata_csv    = "../../datasets/jamendo/metadata/reduced_audio_dataset_15_genres.csv"
audio_folder    = "../../datasets/jamendo/original_audio/"
output_folder   = "../../datasets/jamendo/audio_30_s_chunks/"
output_metadata = "../../datasets/jamendo/metadata/audio_30_s_chunks_metadata.csv"

parallel_chunking_with_threads(
    metadata_csv=metadata_csv,
    audio_folder=audio_folder,
    output_folder=output_folder,
    output_metadata=output_metadata,
    top_db=30,            # docinanie ciszy
    chunk_duration_sec=30,
    max_workers=8
)

Processing: 100%|██████████| 5561/5561 [48:56<00:00,  1.89it/s]  



Zapisano CSV z chunkami: ../../datasets/jamendo/metadata/audio_30_s_chunks_metadata.csv
Liczba chunków: 39097


# Balancing dataset for equal occurences

## Number of chunks

In [12]:
import os
import pandas as pd

# -------------------------------------------
# 1. Funkcja do zliczania wszystkich plików w danym folderze
#    (łącznie z plikami w podfolderach)
# -------------------------------------------
def count_files_in_directory(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            # można dodatkowo filtrować np. czy plik ma rozszerzenie .mp3
            # if file.endswith('.mp3'):
            #     count += 1
            count += 1
    return count

# -------------------------------------------
# PRZYKŁAD UŻYCIA: zliczanie wszystkich chunków (plików) w folderze
# -------------------------------------------
directory_path = "../../datasets/jamendo/audio_30_s_chunks/"
file_count = count_files_in_directory(directory_path)
print(f"Liczba plików (chunków) w '{directory_path}': {file_count}")

Liczba plików (chunków) w '../../datasets/jamendo/audio_30_s_chunks/': 39097


## Number of genre occurrences

In [13]:
# -------------------------------------------
# 2. Wczytanie metadanych chunków
#    Zakładamy, że mamy kolumnę "TAGS" z listą gatunków w formacie "Rock;Pop"
# -------------------------------------------
df = pd.read_csv("../../datasets/jamendo/metadata/audio_30_s_chunks_metadata.csv")

# -------------------------------------------
# 3. Rozbicie kolumny TAGS i obliczenie single/multiple
# -------------------------------------------
# Jeśli kolumna z gatunkami nazywa się inaczej (np. "tags"), dostosuj poniższy kod.
df["TAGS"] = df["TAGS"].astype(str).str.split(';')

# Dodajemy kolumnę z liczbą gatunków
df["genre_count"] = df["TAGS"].apply(len)

# Rozbijamy wiersze tak, by każdy gatunek był w osobnym wierszu
df_expanded = df.explode("TAGS")

# Tworzymy kolumnę 'genre_type' informującą, czy chunk ma 1 gatunek czy wiele
df_expanded["genre_type"] = df_expanded["genre_count"].apply(lambda x: "Single" if x == 1 else "Multiple")

# -------------------------------------------
# 4. Grupowanie wg gatunku i rodzaju
# -------------------------------------------
genre_breakdown = df_expanded.groupby(["TAGS", "genre_type"]).size().unstack(fill_value=0)

print("\nZestawienie liczby chunków wg gatunku i rodzaju (Single/Multiple):")
print(genre_breakdown)


Zestawienie liczby chunków wg gatunku i rodzaju (Single/Multiple):
genre_type   Multiple  Single
TAGS                         
Ambient          1834    2285
Blues             681    1736
Elektronika      3330    2290
Folk             1374    1920
Funk             1264    1947
Hip-Hop           735    2130
House             957    3000
Jazz             1393    2366
Klasyczna         995    2285
Latin             637    1115
Metal             406    3354
Pop              2582     875
Reggae            329    3210
Rock             2144    1305
Techno           1727       0
